In [1]:
import gensim
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [14]:
import pandas as pd
import re
dataset = pd.read_csv('spam_ham_dataset.csv', usecols = ['text','label_num'])

In [15]:
dataset['text'] = dataset['text'].apply(lambda x: re.sub(r'Subject:','',x))
dataset['text'] = dataset['text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+','',x))
dataset['text'] = dataset['text'].apply(lambda x: re.sub(r'[^a-z\s]','',x))

In [16]:
dataset['tokens'] = dataset['text'].str.lower().str.split()

In [17]:
dataset.head()

,text,label_num,tokens
0,enron methanol meter \r\nthis is a follow ...,0,"[enron, methanol, meter, this, is, a, follow, ..."
1,hpl nom for january \r\n see attached file ...,0,"[hpl, nom, for, january, see, attached, file, ..."
2,neon retreat\r\nho ho ho we re around to th...,0,"[neon, retreat, ho, ho, ho, we, re, around, to..."
3,photoshop windows office cheap main trend...,1,"[photoshop, windows, office, cheap, main, tren..."
4,re indian springs\r\nthis deal is to book th...,0,"[re, indian, springs, this, deal, is, to, book..."


In [18]:
#train word2vec model
w2v_model = Word2Vec(sentences=dataset['tokens'], vector_size=100, window=5, min_count=1,workers=3)

In [22]:
import numpy as np
#Function to average word vectors
def vectorize(tokens):
    vectors = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
    return np.mean(vectors,axis=0) if vectors else np.zeros(100)

In [23]:
#create matric of features
x = np.array([vectorize(tokens) for tokens in dataset['tokens']])
y = dataset['label_num'].values
x.shape, y.shape

((5171, 100), (5171,))

In [26]:
#train and test split
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2, random_state=42, stratify=y)
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((4136, 100), (1035, 100), (4136,), (1035,))

In [27]:
# train classifier
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(xtrain, ytrain)
clf.score(xtest, ytest)
 

0.9497584541062802

we can use different ML algorithm to make it more bit good in accuacy but BTW accuracy is optimal at all, it's up to you.